# Projet Huffman

In [2]:
def dico_apparition_par_charactere(texte):
    """
    string -> dict
    retourne le dictionnaire du nombre d'apparitions de chaque caractère
    """
    # Dictionnaire de départ
    _apc = {}
    
    # Pour chaque caractère, on regarde s'il est dans le dictionnaire. 
    # Si oui, on augmente sa valeur
    # Si non, on l'initialise à 1
    for caractère in texte:
        if caractère in _apc:
            _apc[caractère] += 1
        else:
            _apc[caractère] = 1
    return _apc

assert dico_apparition_par_charactere('') == {}
assert dico_apparition_par_charactere('Hello World') == {'H': 1, 'e': 1, 'l': 3, 'o': 2, ' ': 1, 'W': 1, 'r': 1, 'd': 1}
assert dico_apparition_par_charactere('aaabbbcccddd') == {'a': 3, 'b' : 3, 'c': 3, 'd': 3}

## Classe Arbre

Pour répondre au problème, il nous faut un arbre binaire. Mais ici, la structure de l'arbre diffère de celle du cours. En effet, un arbre de Huffman a pour feuilles les lettres du texte auxquelles est associé leur poids. Les autres noeuds parents sont constitués de la somme des poids de leurs enfants.

### Partie théorique : explications

#### Structure

Notre classe Arbre devra donc contenir les attributs suivants :

- Gauche : le fils gauche (ou None)
- Droit : le fils droit (ou None)
- Lettre : Pour une feuille, la lettre représentée ; pour les autres noeuds, les lettres de ses enfants (nous reviendrons par la suite sur ce point)
- Poids : Pour une feuille, le nombre d'apparitions de la lettre ; pour les autres noeuds, la somme du poids de ses enfants.

#### Utilisation et définitions

Voyons maintenant comment utiliser notre arbre, avec l'exemple ci-dessous qui décompose le texte : 'Hello'.

Pour construire un arbre de Huffman, nous devons d'abord partir des feuilles qui sont les différentes lettres, ici : e, o, H, l. Comme ce sont des feuilles, les noeuds représentant ces lettres ont leurs fils à None, et leur poids est leur nombre d'apparitions.

La suite de l'abre est très simple : les deux noeuds aux plus petits poids sont assemblés dans un nouveau noeud. Ici, les deux plus petits noeud sont le 'e' et le 'o'. Ils sont donc rassemblés en tant que fils gauche (le plus fréquent, mais ici leurs poiids sont égaux) et fils droit.

Le nouveau noeud créer à donc ses fils d'assigné. Mais nous ne préciserons pas, lors de sa création, sa lettre et son poids, car ces valeurs seront calculées automatiquement selon l'idée suivante : le poids est calculé en faisant la somme du poids des deux fils ; la lettre se compose de la concaténation des lettres du fils gauche et du fils droit.

Voici un exemple des étapes pour comprendre (toujours avec l'arbre ci-dessous) : 

1. Les noeuds 'e' et 'o' sont assemblés dans un noeud parent.
2. Ce noeud parent se voit attribuer le poids 2 (la somme des poids de 'e' et 'o'.
3. Le noeud parent concatène les lettres de ses fils, et sa lettre devient : 'eo'.
4. On répète ce processus jusqu'à avoir un seul noeud.

#### Exemple : arbre du mot 'Hello'

![Arbre Huffman Exemple](arbre_exemple_Hello.png)

### Partie pratique : implémentation en python

Nous pouvons maintenant créer notre classe python Arbre. Le paramètre l (pour lettre) et p (pour poids) auront une valer par défaut. Ainsi, nous les renseigneront s'il s'agit d'une feuille, autrement ils seront calculés automatiquement. Voici la classe Arbre :

In [36]:
class Arbre:
    
    # Le construceur : 
    # En paramètre (nom complet) : gauche, droite, lettre ('' si non renseigné), poids (-1 si non renseigné)
    # Les attributs sont les mêmes qu'énnoncés plus haut.
    def __init__(self, g, d,  l='', p=-1):
        self.gauche = g
        self.lettre = l
        self.droit = d
        self.poids = p
        
        # Si le poids vaut -1 c'est qu'il n'a pas été renseigné, il faut donc le calculer.
        if p == -1:
            self.calculer_poids()
            
        # Si la lettre vaut '' c'est qu'elle n'a pas été renseignée, il faut donc calculer sa valeur.
        if l == '':
            self.calculer_lettres()

    def fils_gauche(self):
        """
        -> Arbre
        Retourne le fils gauche
        """
        return self.gauche

    def fils_droit(self):
        """
        -> Arbre
        Retourne le fils droit
        """
        return self.droit

    def calculer_poids(self):
        """Calcule le poids d'un noeud (non feuille)"""
        self.poids = 0
        if self.gauche is not None:
            self.poids += self.fils_gauche().donne_poids()

        if self.droit is not None:
            self.poids += self.fils_droit().donne_poids()

    def calculer_lettres(self):
        """Calcule la valeur de la lettre du noeud"""
        if self.fils_gauche() is not None:
            self.lettre += self.fils_gauche().donnee()
        if self.fils_droit() is not None:
            self.lettre += self.fils_droit().donnee()

    def donne_poids(self):
        """
        -> int
        Retourne le poids du noeud
        """
        return self.poids

    def donnee(self):
        """
        -> string
        Retourne la lettre du noeud
        """
        return self.lettre

    def __str__(self):
        """
        -> string
        Retourne une chaîne de caractère pour afficher l'arbre à partir de ce noeud
        """
        if self is None:
            return 'None'
        return '(' + str(self.gauche) + ',' + str(self.droit) + ',' + self.lettre + ',' + str(self.poids) + ')'
    
    def __eq__(self, other):
        """
        -> bool
        Retourne vrai si deux arbres sont identiques, faux sinon
        """
        if self is None and other is None:
            return True
        return self.donne_poids() == other.donne_poids() and self.fils_droit() == other.fils_droit() and self.fils_gauche() == other.fils_gauche()

L'arbre de l'exemple 'Hello' sera créé ainsi :

In [37]:
noeud_e = Arbre(None, None, 'e', 1) # Lettre e avec 1 apparition
noeud_o = Arbre(None, None, 'o', 1) # Lettre o avec 1 apparition
noeud_H = Arbre(None, None, 'H', 1) # Lettre H avec 1 apparition
noeud_l = Arbre(None, None, 'l', 2) # Lettre l avec 2 apparitions

# On ne renseigne pas la lettre et le poids, car leurs valeurs sera calculée automatiquement dans le constructeur
noeud_parent_eo = Arbre(noeud_e, noeud_o)
noeud_parent_eoH = Arbre(noeud_parent_eo, noeud_H)
arbre_eoHl = Arbre(noeud_parent_eoH, noeud_l)

# On appelle la méthode __str__ pour afficher l'arbre
print(arbre_eoHl)

((((None,None,e,1),(None,None,o,1),eo,2),(None,None,H,1),eoH,3),(None,None,l,2),eoHl,5)


## Algorithme

#### A la recherche du minimum

Pour créer un arbre de Huffman, comme énoncé plus haut, nous devons au début de chaque cycle d'étape chercher les 2 noeuds qui on les plus petits poids. Pour se faire, nous allons créer une liste, triée par ordre décroissante, des feuilles de notre future arbre de Huffman. 

Commençons donc par rechercher la lettre maximum. Nous créerons ensuite le tableau.

Notre fonction `item_minimal_dans_dict` va donc, à partir d'un dictionnaire d'apparitions par lettre, retourner la lettre la plus fréquente ainsi que son poids. Nous utilisons pour cela la fonction `dico_apparition_par_charactere` créée plus haut.

In [38]:
def item_maximal_dans_dict(dict_de_lettres):
    """
    dict -> string, int
    Retourne la lettre la plus fréquente ainsi que son poids à partir d'un dictionnaire d'apparations par lettre.
    """
    lettre = ''
    poids_max = 0
    for _lettre, apparition in dict_de_lettres.items():
        if apparition > poids_max:
            poids_max = apparition
            lettre = _lettre
    return lettre, poids_max

dictionnaire = dico_apparition_par_charactere('Hello')
assert item_maximal_dans_dict(dictionnaire) == ('l', 2)

dictionnaire = dico_apparition_par_charactere('e abcd eeeee')
assert item_maximal_dans_dict(dictionnaire) == ('e', 6)

dictionnaire = dico_apparition_par_charactere('')
assert item_maximal_dans_dict(dictionnaire) == ('', 0)

#### Création d'un tableau d'arbre

Maintenant que nous pouvons avoir le caractère le plus fréquent, nous pouvons créer une liste, dans laquelle seront triés par ordre croissant de leurs poids les feuilles de notre futur arbre de Huffman.

Pour cela, utilisons notre classe `Arbre` et créons autant de feuilles que de charactères dans le dictionnaire d'apparitions, en recherchant toujours le maximum. Nous le faisons ici de manière récursive, en supprimant à chaque fois l'élément le plus fréquent du dictionnaire à l'aide de la function `pop()` définie par défaut. 

La magie de l'ordre croissant est possible grâce à l'appel récursif, dans lequel nous ajoutons la valeur `arbre` trouvée (c'est-à-dire la maximale) à la fin du tableau.

In [47]:
def creation_liste_arbres(dict_de_lettres):
    """
    dict -> list
    Retourne une liste d'arbre des lettres du dictionnaire, triée par ordre décroissant du poids.
    """
    
    if len(dict_de_lettres) > 0:
        
        # Recherche de l'élément le plus fréquent
        lettre, poids = item_maximal_dans_dict(dict_de_lettres)

        # Création de la feuille : 
        # - les fils ont pour valeur None (définition d'une feuille)
        # - la lettre est celle trouvée ci-dessus (la plus fréquente)
        # - le poids est la valeur du dictionnaire associée à la lettre (la lettre est la clé, le poids la valeur)
        arbre = Arbre(None, None, lettre, dict_de_lettres[lettre])

        # L'élément le plus fréquent est supprimé
        dict_de_lettres.pop(lettre)

        # Création de la liste
        L = creation_liste_arbres(dict_de_lettres) + [arbre]
        
        # L'élément le plus fréquent (supprimé ci-dessus) est remis dans le dictionnaire
        dict_de_lettres[lettre] = poids
        
        # Retour de la liste
        return L
    else:
        return []

dictionnaire = dico_apparition_par_charactere('Hello')
liste = [Arbre(None, None, 'o',1), Arbre(None, None,'e',1), Arbre(None, None,'H',1), Arbre(None, None,'l',2)]

assert creation_liste_arbres(dico_apparition_par_charactere('Hello')) == liste

#### Création de l'arbre de Huffman

Nous avons maintenant tous les éléments nécessaires pour créer notre arbre de Huffman. Le principe de la création est simple : 

- on cherche les deux feuilles qui ont le plus petit poids, qui sont donc les deux premières du tableau renvoyé par `creation_liste_arbres`
- on les assemble dans un nouvel arbre que l'on ajoute à la liste

In [ ]:
def creation_arbre_huffman(tab_arbre):
    arbre = None

    # arbre initial
    smaller1 = tab_arbre.pop(0)
    smaller2 = tab_arbre.pop(0)
    arbre = Arbre(smaller2, smaller1)
    tab_arbre.append(arbre)

    while len(tab_arbre) > 2:
        arbre1, tab_arbre = supprime_et_retourne_minimal_tab_arbre(tab_arbre)
        arbre2, tab_arbre = supprime_et_retourne_minimal_tab_arbre(tab_arbre)
        _arbre = Arbre(arbre2, arbre1)
        tab_arbre.append(_arbre)

    fd, tab_arbre = supprime_et_retourne_minimal_tab_arbre(tab_arbre)
    fg, tab_arbre = supprime_et_retourne_minimal_tab_arbre(tab_arbre)
    return Arbre(fg, fd)